In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
    "Connection": "keep-alive",
    "Referer": "https://www.amazon.in/",
    "Accept-Encoding": "gzip, deflate, br",
    "Host": "www.amazon.in",
}

In [3]:
url = "https://www.amazon.in/s?k=plum+goodness&crid=3PWA3DMD2PCF9&sprefix=plum+goodness%2Caps%2C223&ref=nb_sb_noss_1"
# Send a GET request
response = requests.get(url, headers=headers)

In [4]:
response

<Response [200]>

In [19]:
def scraper():
    soup = BeautifulSoup(response.content,"html.parser")
    container = soup.find("div",class_ = "s-main-slot s-result-list s-search-results sg-row")
    products = []
    items = container.find_all("div", {"data-component-type": "s-search-result"},recursive = True)
    for item in items:

        # Check if item is sponsored
        is_sponsored = item.find("span", class_="s-label-popover-default")
        if is_sponsored:
            continue  # Skip sponsored items

        # Extract title
        title = item.find("h2", class_="a-size-base-plus a-spacing-none a-color-base a-text-normal")
        title = title.text.strip() if title else None

        discount = None
        discount_span = item.find("span", string=lambda s: s and "off" in s)
        if discount_span:
            discount = discount_span.text.strip()
        else:
            discount = "No Discount"

        # Extract price
        price = item.find("span", class_="a-price-whole")
        price = price.text.strip() if price else None

        rating = None
        rating_icon = item.find("i", class_=lambda value: value and value.startswith("a-icon a-icon-star-small"))
        if rating_icon:
            rating = rating_icon.text.strip()

        # Extract rating
        rating_count = item.find("span", class_="a-size-base s-underline-text")
        rating_count = rating_count.text.strip() if rating else None

        # Extract image URL
        image_tag = item.find("img", class_="s-image")
        image_url = image_tag["src"] if image_tag else None

        # Extract detailed page link
        link_tag = item.find("a", class_="a-link-normal s-line-clamp-3 s-link-style a-text-normal")
        detail_page_link = "https://www.amazon.in" + link_tag["href"] if link_tag else "Not Available
        


        # Append to the product list
        if title:
            products.append({
                "Title": title,
                "Price": price,
                "Rating": rating,
                "Rating_Count": rating_count,
                "Discount": discount,
                "Image_URL": image_url,
                "Page_Link": detail_page_link
            
            })

    return products

In [20]:
data = scraper()
df = pd.DataFrame(data)

array(['Not Available',
       'All, Combination, Sensitive, Oily Acne Prone, Dry, Normal',
       'Plum', 'drop', 'All', 'Black', 'Repairs Split Ends, Smoothening',
       'Rania'], dtype=object)

In [11]:
# Call the scraper function
df = df[~df['Title'].duplicated()]
df.sort_values(by = 'Price', ascending = True).reset_index()
df.to_csv(r"D:\Rohini Personal\Data Analysis\Projects Datasets\TopN Analytics Project\Web Scraping\plum_goodness_amazon.csv", index=False)

In [16]:
import requests
from bs4 import BeautifulSoup

def get_product_volume(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            print(f"Page found: {url}")
            detailed_soup = BeautifulSoup(response.content, "html.parser")

            # Try using a more specific selector for volume
            volume_tag = detailed_soup.find("span", class_="a-size-base po-break-word")
            
            # Sometimes the volume might be inside a different tag, so check if it's None
            if volume_tag:
                volume = volume_tag.text.strip()
                print(f"Volume Found: {volume}")
                return volume
            else:
                print("Volume not found in this page.")
                return "Not Available"
        else:
            print(f"Failed to retrieve {url}. Status code: {response.status_code}")
            return "Not Available"
    except requests.exceptions.RequestException as e:
        print(f"Error occurred while fetching {url}: {str(e)}")
        return "Not Available"

# Example usage
url = "https://www.amazon.in/Plum-Niacinamide-Blemish-Free-Fragrance-Free-Transparent/dp/B09PV4379W/ref=sr_1_6?crid=3PWA3DMD2PCF9&dib=eyJ2IjoiMSJ9.7B7E3BUtEhP9WsZLiWCxVq8U2uQkeC1U7GAjRZDpzZtFqoeUdsiKAFh_ZFF0J7Vu7W-jVr9PtyTpFPmdNx_kXpe6pcU_a8wRMvLzlokImlrmThZATVc14VAA6Tm_A6q9uYmj-WHKAtYbE0cEFs_66DP6PGueQtzUuYYE18JvVLxewL0EDcrSMcP0jW9hWYM_kK3mzWkh897-GsHbFMYU4h1Bj191Y_tvdB01fY_K5fAUJB30sQE1P27aTRfWZ2Cop9jJ-yBTFhH7JGJxVm1ofrIYhlcdHpgcrVrRzrPtZJE8AuGAOehlFBkruwNh5C-iDfR23jsTYRy3OpbLtxtjEVv5c-VrbZa-vZUlWPCnuPvOOZHiU7csLQvBBUWqnhefi8hCic9v_QI-NMhiycsYHiYZjLQ0fCX-hha0elbUbtZDuoQoYGo5kNPmC4vMN3YX.Vlza3iEAzaMM0N5DojyBYd2w2kTz5gK1QpDlHLX1ZDU&dib_tag=se&keywords=plum%2Bgoodness&nsdOptOutParam=true&qid=1738156894&sprefix=plum%2Bgoodness%2Caps%2C223&sr=8-6&th=1"
volume = get_product_volume(url)
print(f"Volume for the product: {volume}")

Failed to retrieve https://www.amazon.in/Plum-Niacinamide-Blemish-Free-Fragrance-Free-Transparent/dp/B09PV4379W/ref=sr_1_6?crid=3PWA3DMD2PCF9&dib=eyJ2IjoiMSJ9.7B7E3BUtEhP9WsZLiWCxVq8U2uQkeC1U7GAjRZDpzZtFqoeUdsiKAFh_ZFF0J7Vu7W-jVr9PtyTpFPmdNx_kXpe6pcU_a8wRMvLzlokImlrmThZATVc14VAA6Tm_A6q9uYmj-WHKAtYbE0cEFs_66DP6PGueQtzUuYYE18JvVLxewL0EDcrSMcP0jW9hWYM_kK3mzWkh897-GsHbFMYU4h1Bj191Y_tvdB01fY_K5fAUJB30sQE1P27aTRfWZ2Cop9jJ-yBTFhH7JGJxVm1ofrIYhlcdHpgcrVrRzrPtZJE8AuGAOehlFBkruwNh5C-iDfR23jsTYRy3OpbLtxtjEVv5c-VrbZa-vZUlWPCnuPvOOZHiU7csLQvBBUWqnhefi8hCic9v_QI-NMhiycsYHiYZjLQ0fCX-hha0elbUbtZDuoQoYGo5kNPmC4vMN3YX.Vlza3iEAzaMM0N5DojyBYd2w2kTz5gK1QpDlHLX1ZDU&dib_tag=se&keywords=plum%2Bgoodness&nsdOptOutParam=true&qid=1738156894&sprefix=plum%2Bgoodness%2Caps%2C223&sr=8-6&th=1. Status code: 500
Volume for the product: Not Available
